In [34]:
import urllib3
from urllib3.exceptions import InsecureRequestWarning

import os
import certifi
import pandas as pd
import requests
import re

from functools import reduce

In [35]:
# 경고 끄기
urllib3.disable_warnings(InsecureRequestWarning)   

startYYYYMM = '201701'
endYYYYMM = '202212'

startYYYYQQ = '2017Q1'
endYYYYQQ = '2022Q4'

startYYYY = '2017'
endYYYY = '2022'

key = 'MTSA07N58C5X8EB4LMJL'

In [36]:
# URL request 함수
def urlRequest(url):
    r = requests.get(url, verify=False)
    if r.status_code == 200:
        jo = r.json()
        returnVal = pd.DataFrame(jo['StatisticSearch']['row'])
    else:
        returnVal = pd.DataFrame()
    return returnVal 

In [37]:
# dataRefine 함수
def dataRefine(dfParam):
    if dfParam.empty:
        returnVal = pd.DataFrame()
    else:
        dfParam['TIME'] = dfParam['TIME'].apply(month_to_quarter)
        dfParam = dfParam[~dfParam['TIME'].str.contains('Unknown')]   
        contains_none = pd.Series(dfParam['ITEM_NAME2'].unique()).astype(str).str.contains('none', case=False, na=False)
        if contains_none.any():    # None'이라는 문자열이 포함되어 있는 경우
            tempDf = dfParam[['STAT_NAME', 'ITEM_NAME1', 'TIME', 'DATA_VALUE', 'UNIT_NAME']].copy() 
            newColName = f"{tempDf['STAT_NAME'].unique()}_{tempDf['ITEM_NAME1'].unique()}" 
        else:
            tempDf = dfParam[['STAT_NAME', 'ITEM_NAME1', 'ITEM_NAME2', 'TIME', 'DATA_VALUE', 'UNIT_NAME']].copy() 
            newColName = f"{tempDf['STAT_NAME'].unique()}_{tempDf['ITEM_NAME1'].unique()}_{tempDf['ITEM_NAME2'].unique()}" 
        tempDf.rename(columns={'DATA_VALUE': newColName}, inplace=True)
        returnVal = tempDf[['TIME', newColName, 'UNIT_NAME']]
    return returnVal

In [38]:
# dataRefineSeperate 함수
# 업종별, 지역별 같이 한단계 더 depth있는 데이터를 정제하기 위한 함수
def dataRefineSeperate(dfParam):
    if dfParam.empty:
        returnVal = pd.DataFrame()
    else:
        dfParam['TIME'] = dfParam['TIME'].apply(month_to_quarter)
        dfParam = dfParam[~dfParam['TIME'].str.contains('Unknown')]
        tempDf = dfParam[['STAT_NAME', 'ITEM_NAME1', 'TIME', 'DATA_VALUE', 'UNIT_NAME']].copy() 
        newColName = f"{tempDf['STAT_NAME'].unique()}"     
        tempDf.rename(columns={'DATA_VALUE': newColName}, inplace=True)
        returnVal = tempDf[['TIME', 'ITEM_NAME1', newColName, 'UNIT_NAME']]
    return returnVal

In [39]:
# 추출 데이터가 월단위 데이터이므로, 이를 Quater 단위로 변환
# 월 → 분기(Q1~Q4)로 변환하는 함수 : 3,6,9,12월 자료만 Quater로 변환하고 나머지는 Unknown 처리
def month_to_quarter(prd):
    year = prd[:4]
    month = prd[-2:]
    if len(prd) == 4:   # 연도 데이터가 넘어온 경우
        quarter = ''
    else: 
        if bool(re.fullmatch(r'Q[1-4]', month)):    # Quater 데이터가 넘어온 경우
            quarter = month
        else:
            if month in ['03']:
                quarter = 'Q1'
            elif month in ['06']:
                quarter = 'Q2'
            elif month in ['09']:
                quarter = 'Q3'
            elif month in ['12']:
                quarter = 'Q4'
            else:
                quarter = 'Unknown'
    return f"{year}{quarter}"

In [40]:
# initQuaterDf 함수
# 데이터마다 시계열이 다르므로, 공통 시계열(2019Q1 ~ 2022Q4) DataFrame을 만들어서 추출된 데이터를 붙임(Left Join) 
# 연도 및 분기 리스트
def initQuaterDf():
    years = range(2018, 2023)  # 2018 ~ 2022
    quarters = ['Q1', 'Q2', 'Q3', 'Q4']
    # PRD_DE 값 생성
    prd_de_list = [f"{year}{q}" for year in years for q in quarters]
    # DataFrame 생성 (열 이름이 'PRD_DE')
    return pd.DataFrame({'TIME': prd_de_list})

In [41]:
def mergeTimeSeriesDataFrame(df_param):
    if df_param.empty:
        returnVal = pd.DataFrame()
    else:
        df = initQuaterDf()
        returnVal = df.merge(df_param, on='TIME', how='left')
    return returnVal

In [42]:
def save_excel_and_csv(df, filename):
    folder_name = 'ecos_output_files'
    # 1. 폴더가 없으면 생성
    os.makedirs(folder_name, exist_ok=True)
    # 2. 저장할 전체 경로 만들기
    excel_path = os.path.join(folder_name, f"{filename}.xlsx")
    csv_path = os.path.join(folder_name, f"{filename}.csv")
     # 3. 파일 저장
    df.to_excel(excel_path, index=False)
    df.to_csv(csv_path, index=False, encoding='utf-8')

In [43]:
urlList = [
    # 환율(원/미국달러, 매매기준율, 평균자료) (exchRate)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/731Y004/M/{startYYYYMM}/{endYYYYMM}/0000001/0000100',
    
    # 시장금리(KORIBOR 3개월) (marketIntRate)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/721Y001/Q/{startYYYYQQ}/{endYYYYQQ}/1025000',

    # 대출금리(예금은행 대출금리, 중소기업대출) (LoanIntRate)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/121Y015/Q/{startYYYYQQ}/{endYYYYQQ}/BECBLB020102',

    # 수출(단위: 천달러) (export)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y011/Q/{startYYYYQQ}/{endYYYYQQ}/FIEE',
    
    # 수입(단위: 천달러) (import)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y012/Q/{startYYYYQQ}/{endYYYYQQ}/FIEF',
    
    # 경상수지(계절조정)(단위 : 백만달러) (curAccount)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/301Y017/M/{startYYYYMM}/{endYYYYMM}/SA000',
    
    # 본원통화(M0) (단위 : 십억원) (m0Currency)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/102Y004/M/{startYYYYMM}/{endYYYYMM}/ABA1',
    
    # 협의통화(M1)(단위: 십억원) (m1Currency)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/101Y018/M/{startYYYYMM}/{endYYYYMM}/BBLS00',
    
    # 광의통화(M2)(단위: 십억원) (m2Currency)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/101Y003/M/{startYYYYMM}/{endYYYYMM}/BBHS00',
    
    # 가계대출(단위: 십억원) (householdLoan)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/151Y004/Q/{startYYYYQQ}/{endYYYYQQ}/1100000',
    
    # 가계대출(주담대)(단위: 십억원) (houseLoan)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/151Y004/Q/{startYYYYQQ}/{endYYYYQQ}/11000A0',
    
    # 가계대출(기타)(단위: 십억원) (etcLoan)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/151Y004/Q/{startYYYYQQ}/{endYYYYQQ}/11000B0',
    
    # 8.5.3.소매업태별 판매액지수-전문소매업-경상지수 (단위 2020=100) (retailSalesIdx)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y098/M/{startYYYYMM}/{endYYYYMM}/I74Q/I74A',
    
    # 소비자물가지수 (CPI)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y009/Q/{startYYYYQQ}/{endYYYYQQ}/0',
    
    # 생산자물가지수 (PPI)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/404Y014/Q/{startYYYYQQ}/{endYYYYQQ}/*AA',
    
    # 수출물가지수 (expPriceIdx)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/402Y014/Q/{startYYYYQQ}/{endYYYYQQ}/*AA/C',
    
    # 수입물가지수 (impPriceIdx)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/401Y015/Q/{startYYYYQQ}/{endYYYYQQ}/*AA/C',
    
    # 소비자동향조사(현재생활형편CSI) (curLifCSI)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/511Y002/M/{startYYYYMM}/{endYYYYMM}/FMAA/99988',
    
    # 소비자동향조사(생활형편전망CSI) (predLifeCSI)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/511Y002/M/{startYYYYMM}/{endYYYYMM}/FMBA/99988',
    
    # 소비자동향조사(현재경기판단CSI) (curEconoCSI)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/511Y002/M/{startYYYYMM}/{endYYYYMM}/FMAB/99988',
    
    # 소비자동향조사(향후경기전망CSI) (predEconoCSI)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/511Y002/M/{startYYYYMM}/{endYYYYMM}/FMBB/99988',
    
    # 소비자동향조사(가계수입전망CSI) (predHouseIncomeCSI)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/511Y002/M/{startYYYYMM}/{endYYYYMM}/FMCA/99988',
    
    # 소비자동향조사(소비지출전망CSI) (predConExpCSI)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/511Y002/M/{startYYYYMM}/{endYYYYMM}/FMCB/99988',
    
    # 소비자동향조사(소비자심리지수CSI) (consumerMentCSI)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/511Y002/M/{startYYYYMM}/{endYYYYMM}/FME/99988',
    
    # 경기종합지수(선행지수순환변동치) (preCompIdx)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y067/M/{startYYYYMM}/{endYYYYMM}/I16E',
    
    # 경기종합지수(동행지수순환변동치) (accompCompIdx)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y067/M/{startYYYYMM}/{endYYYYMM}/I16D',
    
    # 경제심리지수(경제심리지수, 순환변동치) (econoMentIdx)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/513Y001/M/{startYYYYMM}/{endYYYYMM}/E2000',
    
    # 도소매업 재고액지수 (wholesaleStkIdx)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y101/Q/{startYYYYQQ}/{endYYYYQQ}/G/T3',

    # 가계 최종소비지출(단위 : 십억원) (houseFinExpend)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y141/Q/{startYYYYQQ}/{endYYYYQQ}/10116'
]

df_dict = {}
for i, url in enumerate(urlList):
    df_dict[f"fd_{i}"] = mergeTimeSeriesDataFrame(dataRefine(urlRequest(url)))

trimmed_dict = {
    key: df.iloc[:, :2]  # 앞의 두 컬럼 선택
    for key, df in df_dict.items()
}

# 딕셔너리에 원소로 되어있는 DataFrame Join
ecosCommon_Q = reduce(lambda left, right: pd.merge(left, right, on=['TIME']), trimmed_dict.values()).sort_values(by=['TIME']).reset_index(drop=True)
ecosCommon_Q.columns = [
    'TIME', 
    'exchRate', 'marketIntRate',
    'LoanIntRate', 'export', 'import', 'curAccount',
    'm0Currency', 'm1Currency', 'm2Currency', 'householdLoan', 'houseLoan', 'etcLoan', 'retailSalesIdx',
    'CPI', 'PPI', 'expPriceIdx', 'impPriceIdx', 
    'curLifCSI', 'predLifeCSI', 'curEconoCSI', 'predEconoCSI', 'predHouseIncomeCSI', 'predConExpCSI', 'consumerMentCSI', 
    'preCompIdx', 'accompCompIdx', 'econoMentIdx', 'wholesaleStkIdx', 'houseFinExpend'
]
#display(ecosCommon_Q)

# 선행지수 만들기 : 3/6/9개월 선행지수를 만들고, (_3M, _6M, _9M)가 붙은 컬럼을 만듦 
columns = list(ecosCommon_Q.columns)
# lag 생성
for col in ecosCommon_Q.columns[1:]:
    # 현재 컬럼의 위치 index
    idx = ecosCommon_Q.columns.get_loc(col) + 1  
    for lag in range(1, 4):  # lag_1 ~ lag_3
        lag_col_name = f"{col}_{lag*3}M"
        ecosCommon_Q.insert(loc=idx, column=lag_col_name, value=ecosCommon_Q[col].shift(lag))
        idx += 1  # 다음 lag 컬럼은 그 다음에 삽입

# TIME컬럼 중 'Q4'로 끝나는 row만 남기고, TIME컬럼의 'Q4'를 모두 지워서 'YYYY'형태의 데이터만 남김
# 'Q4'로 끝나는 row만 필터링
df_q4 = ecosCommon_Q[ecosCommon_Q['TIME'].str.endswith('Q4')].copy()
df_q4['TIME'] = df_q4['TIME'].str.replace('Q4', '', regex=False)
ecosCommon = df_q4.copy()
display(ecosCommon)

,TIME,exchRate,exchRate_3M,exchRate_6M,exchRate_9M,marketIntRate,marketIntRate_3M,marketIntRate_6M,marketIntRate_9M,LoanIntRate,...,econoMentIdx_6M,econoMentIdx_9M,wholesaleStkIdx,wholesaleStkIdx_3M,wholesaleStkIdx_6M,wholesaleStkIdx_9M,houseFinExpend,houseFinExpend_3M,houseFinExpend_6M,houseFinExpend_9M
3,2018,1122.9,1120.6,1092.8,1071.89,1.82,1.67,1.66,1.64,3.88,...,98.8,100.2,NaN,NaN,NaN,NaN,243927.5,242167.4,241148.7,240210
7,2019,1175.84,1197.55,1175.62,1130.72,1.49,1.53,1.79,1.87,3.51,...,95,95.4,NaN,NaN,NaN,NaN,250478,247560.5,245565.2,245453
11,2020,1095.13,1178.8,1210.01,1220.09,0.69,0.62,0.87,1.31,2.86,...,73.9,78,96.7,100.3,100.6,102.3,234997.4,237250.9,235162.1,235008
15,2021,1183.7,1169.54,1121.3,1131.02,1.2,0.82,0.68,0.74,3.11,...,105.3,99.6,99,97.2,94.1,95.9,250816.1,245742.2,244576.6,237449.4
19,2022,1296.22,1391.59,1277.35,1221.03,3.87,2.72,1.8,1.46,5.19,...,101.9,105.4,102.7,101.8,100.1,97.7,258256.6,258558.1,255219.5,247381.5


In [44]:
urlList = [
    # 국내총생산(명목) (GDP)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y101/A/{startYYYY}/{endYYYY}/10101',
    
    # 국민총소득(명목) (GNI)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y101/A/{startYYYY}/{endYYYY}/10102',
    
    # 국민처분가능소득(명목) (NDI)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y101/A/{startYYYY}/{endYYYY}/10104',
    
    # 국민총처분가능소득(명목) (GNDI)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y101/A/{startYYYY}/{endYYYY}/10105',
    
    # 가계총처분가능소득(명목) (GDI)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y101/A/{startYYYY}/{endYYYY}/101051',
    
    # 1인당 국내총생산(명목) (PGDP)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y101/A/{startYYYY}/{endYYYY}/10107',
    
    # 1인당 국민총소득(명목) (PGNI)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y101/A/{startYYYY}/{endYYYY}/10106',
    
    # 1인당 가계총처분가능소득(명목) (PGDI)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y101/A/{startYYYY}/{endYYYY}/1010602'
]

df_dict = {}
for i, url in enumerate(urlList):
    df_dict[f"fd_{i}"] = dataRefine(urlRequest(url))

trimmed_dict = {
    key: df.iloc[:, :2]  # 앞의 두 컬럼 선택
    for key, df in df_dict.items()
}

# 딕셔너리에 원소로 되어있는 DataFrame Join
ecosGDP_Y = reduce(lambda left, right: pd.merge(left, right, on=['TIME']), trimmed_dict.values()).sort_values(by=['TIME']).reset_index(drop=True)
ecosGDP_Y.columns = [
    'TIME', 
    'GDP', 'GNI', 'NDI', 'GNDI', 'GDI', 'PGDP', 'PGNI', 'PGDI'
]
#display(ecosGDP_Y)

# 선행지수 만들기 : 1년 선행지수를 만들고, (_1Y)가 붙은 컬럼을 만듦 
columns = list(ecosGDP_Y.columns)
# lag 생성
for col in ecosGDP_Y.columns[1:]:
    # 현재 컬럼의 위치 index
    idx = ecosGDP_Y.columns.get_loc(col) + 1
    lag_col_name = f"{col}_1Y"
    ecosGDP_Y.insert(loc=idx, column=lag_col_name, value=ecosGDP_Y[col].shift(1))
    idx += 1  # 다음 lag 컬럼은 그 다음에 삽입

ecosGDP = ecosGDP_Y.copy()
ecosGDP = ecosGDP[ecosGDP['TIME'] > startYYYY]
display(ecosGDP)

,TIME,GDP,GDP_1Y,GNI,GNI_1Y,NDI,NDI_1Y,GNDI,GNDI_1Y,GDI,GDI_1Y,PGDP,PGDP_1Y,PGNI,PGNI_1Y,PGDI,PGDI_1Y
1,2018,2006974.5,1934233.9,2014588.5,1941716.5,1628018.5,1577031.4,2007352.8,1935277.5,1093458.4,1046674.9,3890.6,3765.9,3905.4,3780.5,2119.7,2037.8
2,2019,2040594.3,2006974.5,2057204.1,2014588.5,1648030.4,1628018.5,2051307.9,2007352.8,1141898.9,1093458.4,3942,3890.6,3974.1,3905.4,2205.9,2119.7
3,2020,2058466.5,2040594.3,2075410.3,2057204.1,1648918.5,1648030.4,2072253.6,2051307.9,1181885.9,1141898.9,3971.1,3942,4003.8,3974.1,2280,2205.9
4,2021,2221912.9,2058466.5,2245326.5,2075410.3,1780892.1,1648918.5,2240839.7,2072253.6,1224043.6,1181885.9,4291.9,3971.1,4337.2,4003.8,2364.4,2280
5,2022,2323781.5,2221912.9,2351837,2245326.5,1840003,1780892.1,2349203.9,2240839.7,1288946.8,1224043.6,4497.1,4291.9,4551.4,4337.2,2494.5,2364.4


In [45]:
ecosCommon = pd.merge(ecosCommon, ecosGDP, on='TIME', how='inner')
save_excel_and_csv(ecosCommon, 'ecos공통')

In [46]:
# 경제활동별 GDP 및 GNI(계절조정, 실질, 분기) (단위: 십억원)

url_dict = {
    # 경제활동별 GDP 및 GNI(계절조정, 실질, 분기) (GDPGNI) - 임대업(N76)은 없음
    'gdpgni' :  [
        # 경제활동별 GDP 및 GNI(계절조정, 실질, 분기) : 도소매업(G)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y103/Q/{startYYYYQQ}/{endYYYYQQ}/110601',
        
        # 경제활동별 GDP 및 GNI(계절조정, 실질, 분기) : 숙박 및 음식점업(I)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y103/Q/{startYYYYQQ}/{endYYYYQQ}/110602',
        
        # 경제활동별 GDP 및 GNI(계절조정, 실질, 분기) : 운수업(H)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y103/Q/{startYYYYQQ}/{endYYYYQQ}/1107',
        
        # 경제활동별 GDP 및 GNI(계절조정, 실질, 분기) : 금융보험업(K)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y103/Q/{startYYYYQQ}/{endYYYYQQ}/1108',
        
        # 경제활동별 GDP 및 GNI(계절조정, 실질, 분기) : 부동산업(L)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y103/Q/{startYYYYQQ}/{endYYYYQQ}/1109',
        
        # 경제활동별 GDP 및 GNI(계절조정, 실질, 분기) : 사업지원서비스업(N75)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y103/Q/{startYYYYQQ}/{endYYYYQQ}/111502',
        
        # 경제활동별 GDP 및 GNI(계절조정, 실질, 분기) : 교육서비스업(P)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y103/Q/{startYYYYQQ}/{endYYYYQQ}/1111',
        
        # 경제활동별 GDP 및 GNI(계절조정, 실질, 분기) : 의료, 보건업 및 사회복지서비스업(Q)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y103/Q/{startYYYYQQ}/{endYYYYQQ}/1112',
        
        # 경제활동별 GDP 및 GNI(계절조정, 실질, 분기) : 예술, 스포츠 및 여가관련 서비스업(R)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y103/Q/{startYYYYQQ}/{endYYYYQQ}/11131',
        
        # 경제활동별 GDP 및 GNI(계절조정, 실질, 분기) : 기타 서비스업(S)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y103/Q/{startYYYYQQ}/{endYYYYQQ}/11131'
    ],
    # 경제활동별 GDP 성장기여도(GDP에 대한 성장기여도)
    'growthContribution' :  [
        # 경제활동별 GDP 성장기여도 : 도소매업(G)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y123/Q/{startYYYYQQ}/{endYYYYQQ}/110601',
        
        # 경제활동별 GDP 성장기여도 : 숙박 및 음식점업(I)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y123/Q/{startYYYYQQ}/{endYYYYQQ}/110602',
        
        # 경제활동별 GDP 성장기여도 : 운수업(H)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y123/Q/{startYYYYQQ}/{endYYYYQQ}/1107',
        
        # 경제활동별 GDP 성장기여도 : 금융보험업(K)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y123/Q/{startYYYYQQ}/{endYYYYQQ}/1107',
        
        # 경제활동별 GDP 성장기여도 : 부동산업(L)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y123/Q/{startYYYYQQ}/{endYYYYQQ}/1109',
        
        # 경제활동별 GDP 성장기여도 : 사업지원서비스업(N75)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y123/Q/{startYYYYQQ}/{endYYYYQQ}/111502',
        
        # 경제활동별 GDP 성장기여도 : 교육서비스업(P)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y123/Q/{startYYYYQQ}/{endYYYYQQ}/1111',
        
        # 경제활동별 GDP 성장기여도 : 의료, 보건업 및 사회복지서비스업(Q)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y123/Q/{startYYYYQQ}/{endYYYYQQ}/1112',
        
        # 경제활동별 GDP 성장기여도 : 예술, 스포츠 및 여가관련 서비스업(R)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y123/Q/{startYYYYQQ}/{endYYYYQQ}/11131',

        # 경제활동별 GDP 성장기여도 : 기타 서비스업(S)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/200Y123/Q/{startYYYYQQ}/{endYYYYQQ}/11132'
    ]
}

merged_dict = {}
for urlDictKey, urlList in url_dict.items():
    df_dict = {}
    for i, url in enumerate(urlList):
        df_dict[f"fd_{i}"] = dataRefineSeperate(urlRequest(url))
    merged_dict[f'{urlDictKey}'] = pd.concat(df_dict.values(), axis=0)  # 세로로 합치기 (row-wise)

trimmed_dict = {
    key: df.iloc[:, :3]  # 앞의 세 컬럼 선택
    for key, df in merged_dict.items()
}

# 딕셔너리에 원소로 되어있는 DataFrame Join
induGDPGNI_Q = reduce(lambda left, right: pd.merge(left, right, on=['TIME', 'ITEM_NAME1']), trimmed_dict.values()).sort_values(by=['TIME', 'ITEM_NAME1']).reset_index(drop=True)
#display(induGDPGNI_Q)
induGDPGNI_Q.columns = [
    'TIME', 'KSIC', 'GDPGNI', 'GDPgCont'
]
#display(induGDPGNI_Q)

# 선행지수 만들기 : 3/6/9개월 선행지수를 만들고, (_3M, _6M, _9M)가 붙은 컬럼을 만듦 
columns = list(induGDPGNI_Q.columns)
# lag 생성
for col in induGDPGNI_Q.columns[2:]:
    # 현재 컬럼의 위치 index
    idx = induGDPGNI_Q.columns.get_loc(col) + 1  
    for lag in range(1, 4):  # lag_1 ~ lag_3
        lag_col_name = f"{col}_{lag*3}M"
        induGDPGNI_Q.insert(loc=idx, column=lag_col_name, value=induGDPGNI_Q[col].shift(lag))
        idx += 1  # 다음 lag 컬럼은 그 다음에 삽입

# TIME컬럼 중 'Q4'로 끝나는 row만 남기고, TIME컬럼의 'Q4'를 모두 지워서 'YYYY'형태의 데이터만 남김
# 'Q4'로 끝나는 row만 필터링
df_q4 = induGDPGNI_Q[induGDPGNI_Q['TIME'].str.endswith('Q4')].copy()
df_q4['TIME'] = df_q4['TIME'].str.replace('Q4', '', regex=False)
induGDPGNI = df_q4.copy()
replace_dict = {
    '도소매업': 'G',
    '운수업': 'H',
    '숙박 및 음식점업': 'I',
    '금융 및 보험업': 'K',
    '부동산업': 'L',
    '사업지원 서비스업': 'N75',
    '임대업': 'N76',
    '교육서비스업': 'P',
    '의료, 보건업 및 사회복지서비스업': 'Q',
    '예술, 스포츠 및 여가관련 서비스업': 'R',
    '기타 서비스업': 'S'
}
induGDPGNI['KSIC'] = induGDPGNI['KSIC'].replace(replace_dict)
induGDPGNI = induGDPGNI[induGDPGNI['TIME'] > startYYYY]
display(induGDPGNI)
save_excel_and_csv(induGDPGNI, 'ecos산업_GDPGNI')

,TIME,KSIC,GDPGNI,GDPGNI_3M,GDPGNI_6M,GDPGNI_9M,GDPgCont,GDPgCont_3M,GDPgCont_6M,GDPgCont_9M
70,2018,P,22042.7,23263,16810.8,16810.8,0,0.1,0,0
71,2018,G,43597.2,22042.7,23263,16810.8,0.1,0,0.1,0
72,2018,L,37266.8,43597.2,22042.7,23263,0,0.1,0,0.1
73,2018,N75,16167.8,37266.8,43597.2,22042.7,0,0,0.1,0
74,2018,I,12168.2,16167.8,37266.8,43597.2,0,0,0,0.1
75,2018,R,6576.2,12168.2,16167.8,37266.8,0,0,0,0
76,2018,R,6576.2,6576.2,12168.2,16167.8,0,0,0,0
77,2018,H,17780.6,6576.2,6576.2,12168.2,0.1,0,0,0
78,2018,H,17780.6,17780.6,6576.2,6576.2,0.1,0.1,0,0
79,2018,Q,24006.6,17780.6,17780.6,6576.2,0.1,0.1,0.1,0


In [47]:
# 중소기업 매출채권회전율 (단위 : 회, receivTurnover) - 금융 보험업 없음

urlList = [
    # 중소기업 매출채권회전율(도매및소매업, G)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/501Y008/A/{startYYYY}/{endYYYY}/G/M/808',
    
    # 중소기업 매출채권회전율(운수및창고업, H)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/501Y008/A/{startYYYY}/{endYYYY}/H/M/808',
    
    # 중소기업 매출채권회전율(숙박및음식점업, I)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/501Y008/A/{startYYYY}/{endYYYY}/I/M/808',
    
    # 중소기업 매출채권회전율(부동산업, L)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/501Y008/A/{startYYYY}/{endYYYY}/L/M/808',
    
    # 중소기업 매출채권회전율(사업시설관리 및 사업지원 및 임대서비스업, N)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/501Y008/A/{startYYYY}/{endYYYY}/N/M/808',
    
    # 중소기업 매출채권회전율(교육서비스업, P)
     f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/501Y008/A/{startYYYY}/{endYYYY}/P/M/808',
    
    # 중소기업 매출채권회전율(예술스포츠 및 여가관련 서비스업, R)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/501Y008/A/{startYYYY}/{endYYYY}/R/M/808'
]

def innerDataRefine(dfParam):
    dfParam['TIME'] = dfParam['TIME'].apply(month_to_quarter)
    dfParam = dfParam[~dfParam['TIME'].str.contains('Unknown')]
    tempDf = dfParam[['STAT_NAME', 'ITEM_NAME1', 'ITEM_NAME2', 'ITEM_NAME3', 'TIME', 'DATA_VALUE', 'UNIT_NAME']].copy() 
    newColName = f"{tempDf['STAT_NAME'].unique()}_{tempDf['ITEM_NAME2'].unique()}_{tempDf['ITEM_NAME3'].unique()}"     
    tempDf.rename(columns={'DATA_VALUE': newColName}, inplace=True)
    return tempDf[['TIME', 'ITEM_NAME1', newColName, 'UNIT_NAME']]

df_dict = {}
for i, url in enumerate(urlList):
    df_dict[f"fd_{i}"] = innerDataRefine(urlRequest(url))

receivablesTurnover_Y = pd.concat(df_dict.values(), axis=0).iloc[:, :3].sort_values(by=['TIME', 'ITEM_NAME1']).reset_index(drop=True)
receivablesTurnover_Y.columns = [
    'TIME', 'KSIC', 'receivTurnover' 
]
#display(receivablesTurnover_Y)

# 선행지수 만들기 : 1년 선행지수를 만들고, (_1Y)가 붙은 컬럼을 만듦 
columns = list(receivablesTurnover_Y.columns)
# lag 생성
for col in receivablesTurnover_Y.columns[2:]:
    # 현재 컬럼의 위치 index
    idx = receivablesTurnover_Y.columns.get_loc(col) + 1
    lag_col_name = f"{col}_1Y"
    receivablesTurnover_Y.insert(loc=idx, column=lag_col_name, value=receivablesTurnover_Y[col].shift(1))
    idx += 1  # 다음 lag 컬럼은 그 다음에 삽입

receivablesTurnover = receivablesTurnover_Y.copy()
receivablesTurnover['KSIC'] = receivablesTurnover['KSIC'].str[0]
receivablesTurnover = receivablesTurnover[receivablesTurnover['TIME'] > startYYYY]
display(receivablesTurnover)
save_excel_and_csv(receivablesTurnover, 'ecos산업_중소기업_매출채권회전율')

,TIME,KSIC,receivTurnover,receivTurnover_1Y
7,2018,G,7.92,28.39
8,2018,H,11.39,7.92
9,2018,I,21.54,11.39
10,2018,L,8.43,21.54
11,2018,N,12.59,8.43
12,2018,P,41.24,12.59
13,2018,R,25.17,41.24
14,2019,G,7.91,25.17
15,2019,H,11.45,7.91
16,2019,I,26.66,11.45


In [48]:
# 산업별 서비스업생산지수(계절조정지수, 단위 : 2020=100) : serviceProdIdx
urlList = [
    # 산업별 서비스업생산지수(도매 및 소매업)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y038/Q/{startYYYYQQ}/{endYYYYQQ}/I51AB/3',
        
    # 산업별 서비스업생산지수(운수 및 창고업)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y038/Q/{startYYYYQQ}/{endYYYYQQ}/I51AC/3',
        
    # 산업별 서비스업생산지수(숙박 및 음식점업)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y038/Q/{startYYYYQQ}/{endYYYYQQ}/I51AD/3',
        
    # 산업별 서비스업생산지수(부동산업)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y038/Q/{startYYYYQQ}/{endYYYYQQ}/I51AG/3',
        
    # 산업별 서비스업생산지수(사업시설관리, 사업지원 및 임대서비스업)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y038/Q/{startYYYYQQ}/{endYYYYQQ}/I51AI/3',
        
    # 산업별 서비스업생산지수(교육서비스업)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y038/Q/{startYYYYQQ}/{endYYYYQQ}/I51AJ/3',
        
    # 산업별 서비스업생산지수(보건 및 사회복지 서비스업)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y038/Q/{startYYYYQQ}/{endYYYYQQ}/I51AK/3',
    
    # 산업별 서비스업생산지수(예술, 소프츠 및 여기관련 서비스업)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y038/Q/{startYYYYQQ}/{endYYYYQQ}/I51AL/3',
    
    # 산업별 서비스업생산지수(협회 및 단체, 수리 및 기타 개인서비스업)
    f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y038/Q/{startYYYYQQ}/{endYYYYQQ}/I51AM/3'
]

df_dict = {}
for i, url in enumerate(urlList):
    df_dict[f"fd_{i}"] = dataRefineSeperate(urlRequest(url))

induServiceProdIndex_Q = pd.concat(df_dict.values(), axis=0).iloc[:, :3]  # 세로로 합치기 (row-wise)
induServiceProdIndex_Q.columns = [
    'TIME', 'KSIC', 'serviceProdIdx' 
]
#display(induServiceProdIndex_Q)

# 선행지수 만들기 : 3/6/9개월 선행지수를 만들고, (_3M, _6M, _9M)가 붙은 컬럼을 만듦 
columns = list(induServiceProdIndex_Q.columns)
# lag 생성
for col in induServiceProdIndex_Q.columns[2:]:
    # 현재 컬럼의 위치 index
    idx = induServiceProdIndex_Q.columns.get_loc(col) + 1  
    for lag in range(1, 4):  # lag_1 ~ lag_3
        lag_col_name = f"{col}_{lag*3}M"
        induServiceProdIndex_Q.insert(loc=idx, column=lag_col_name, value=induServiceProdIndex_Q[col].shift(lag))
        idx += 1  # 다음 lag 컬럼은 그 다음에 삽입

# TIME컬럼 중 'Q4'로 끝나는 row만 남기고, TIME컬럼의 'Q4'를 모두 지워서 'YYYY'형태의 데이터만 남김
# 'Q4'로 끝나는 row만 필터링
df_q4 = induServiceProdIndex_Q[induServiceProdIndex_Q['TIME'].str.endswith('Q4')].copy()
df_q4['TIME'] = df_q4['TIME'].str.replace('Q4', '', regex=False)
induServiceProdIndex = df_q4.copy()
replace_dict = {
    '도매 및 소매업': 'G',
    '운수 및 창고업': 'H',
    '숙박 및 음식점업': 'I',
    '금융 및 보험업': 'K',
    '부동산업': 'L',
    '사업시설관리, 사업지원 및 임대 서비스업': 'N',
    '교육 서비스업': 'P',
    '보건 및 사회복지서비스업': 'Q',
    '예술, 스포츠 및 여가관련 서비스업': 'R',
    '협회 및 단체, 수리 및 기타개인서비스업': 'S'
}
induServiceProdIndex['KSIC'] = induServiceProdIndex['KSIC'].replace(replace_dict)
induServiceProdIndex = induServiceProdIndex[induServiceProdIndex['TIME'] > startYYYY]
display(induServiceProdIndex)
save_excel_and_csv(induServiceProdIndex, 'ecos산업_서비스업생산지수')

,TIME,KSIC,serviceProdIdx,serviceProdIdx_3M,serviceProdIdx_6M,serviceProdIdx_9M
7,2018,G,103.5,102.9,103.5,103.3
11,2019,G,102.8,102.9,102.8,102.7
15,2020,G,102.3,100.1,98.1,99.4
19,2021,G,106,104.9,103.7,103
23,2022,G,107.3,108.5,106.8,106.1
7,2018,H,116.3,117.1,118.1,114.5
11,2019,H,115.4,117.5,116.2,116.4
15,2020,H,99.1,96.9,92.8,111.2
19,2021,H,110.4,103.4,102.8,99.9
23,2022,H,131.6,127.9,121.9,111.5


In [49]:
# 주택매매가격지수, 주택전세가격지수 (단위: 2022.01=100)
url_dict = {
    'buy' :  [
        # 주택매매가격지수(전국) : (kbHouseSaleIdx)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y062/M/{startYYYYMM}/{endYYYYMM}/P63A',

        # 주택매매가격지수(서울)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y062/M/{startYYYYMM}/{endYYYYMM}/P63AD',
    ],
    'lease' :  [
        # 주택전세가격지수(전국) : (kbHouseLentIdx)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y063/M/{startYYYYMM}/{endYYYYMM}/P64A',

        # 주택전세가격지수(서울)
        f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/901Y063/M/{startYYYYMM}/{endYYYYMM}/P64AD'
    ]
}

merged_dict = {}
for urlDictKey, urlList in url_dict.items():
    df_dict = {}
    for i, url in enumerate(urlList):
        df_dict[f"fd_{i}"] = dataRefineSeperate(urlRequest(url))
    merged_dict[f'{urlDictKey}'] = pd.concat(df_dict.values(), axis=0)  # 세로로 합치기 (row-wise)

trimmed_dict = {
    key: df.iloc[:, :3]  # 앞의 세 컬럼 선택
    for key, df in merged_dict.items()
}

# 딕셔너리에 원소로 되어있는 DataFrame Join
houseBuyLendIndex_Q = reduce(lambda left, right: pd.merge(left, right, on=['TIME', 'ITEM_NAME1']), trimmed_dict.values()).sort_values(by=['TIME', 'ITEM_NAME1']).reset_index(drop=True)
houseBuyLendIndex_Q.columns = [
    'TIME', 'REGION', 'kbHouseSaleIdx', 'kbHouseLentIdx' 
]
#display(houseBuyLendIndex_Q)

# 선행지수 만들기 : 3/6/9개월 선행지수를 만들고, (_3M, _6M, _9M)가 붙은 컬럼을 만듦 
columns = list(houseBuyLendIndex_Q.columns)
# lag 생성
for col in houseBuyLendIndex_Q.columns[2:]:
    # 현재 컬럼의 위치 index
    idx = houseBuyLendIndex_Q.columns.get_loc(col) + 1  
    for lag in range(1, 4):  # lag_1 ~ lag_3
        lag_col_name = f"{col}_{lag*3}M"
        houseBuyLendIndex_Q.insert(loc=idx, column=lag_col_name, value=houseBuyLendIndex_Q[col].shift(lag))
        idx += 1  # 다음 lag 컬럼은 그 다음에 삽입

# TIME컬럼 중 'Q4'로 끝나는 row만 남기고, TIME컬럼의 'Q4'를 모두 지워서 'YYYY'형태의 데이터만 남김
# 'Q4'로 끝나는 row만 필터링
df_q4 = houseBuyLendIndex_Q[houseBuyLendIndex_Q['TIME'].str.endswith('Q4')].copy()
df_q4['TIME'] = df_q4['TIME'].str.replace('Q4', '', regex=False)
houseBuyLendIndex = df_q4.copy()
replace_dict = {
    '총지수': '전국',
    '총지수(서울)': '서울특별시'
}
houseBuyLendIndex['REGION'] = houseBuyLendIndex['REGION'].replace(replace_dict)
houseBuyLendIndex = houseBuyLendIndex[houseBuyLendIndex['TIME'] > startYYYY]
display(houseBuyLendIndex)
save_excel_and_csv(houseBuyLendIndex, 'ecos지역_주택매매전세가격지수')

,TIME,REGION,kbHouseSaleIdx,kbHouseSaleIdx_3M,kbHouseSaleIdx_6M,kbHouseSaleIdx_9M,kbHouseLentIdx,kbHouseLentIdx_3M,kbHouseLentIdx_6M,kbHouseLentIdx_9M
14,2018,전국,79.861,76.572,79.232,73.426,86.01,81.41,85.903,80.533
15,2018,서울특별시,78.098,79.861,76.572,79.232,81.958,86.01,81.41,85.903
22,2019,전국,80.054,78.957,79.589,78.158,85.576,81.869,85.331,81.579
23,2019,서울특별시,80.129,80.054,78.957,79.589,82.482,85.576,81.869,85.331
30,2020,전국,86.735,85.383,83.798,81.755,91.176,86.254,87.962,83.439
31,2020,서울특별시,88.701,86.735,85.383,83.798,90.851,91.176,86.254,87.962
38,2021,전국,99.72,97.624,97.123,94.084,99.728,98.147,97.976,95.394
39,2021,서울특별시,99.787,99.72,97.624,97.123,99.765,99.728,98.147,97.976
46,2022,전국,97.894,100.641,100.568,100.723,97.279,100.651,100.773,100.723
47,2022,서울특별시,98.519,97.894,100.641,100.568,96.969,97.279,100.651,100.773
